<a href="https://colab.research.google.com/github/lucprosa/dataeng-basic-course/blob/main/spark_streaming/examples/coinbase_producer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Producer Coinbase

- To be executed in Google Colab
- Connect to GCLOUD
- Fetch data from Coinbase via WebSocket
- Copy data to GC bucket

In [ ]:
from google.colab import auth
auth.authenticate_user()

project_id = 'data-eng-dev-437916'
!gcloud config set project {project_id}

In [ ]:
!pip install websocket-client

In [ ]:
!rm -r /content/btc_stream
!mkdir -p /content/btc_stream/landing

In [ ]:
import websocket
import json
import os
import time
from datetime import datetime

OUTPUT_DIR = "/content/btc_stream/landing"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def on_message(ws, message):
    data = json.loads(message)
    if data.get("type") == "ticker":
        timestamp = datetime.utcnow().strftime("%Y%m%d%H%M%S%f")
        filename = f"{OUTPUT_DIR}/btc_{timestamp}.json"
        with open(filename, "w") as f:
            json.dump(data, f)
        time.sleep(5)
        landing_path=f"gs://edit-data-eng-dev/datalake/landing/btc/btc_{timestamp}.json"
        !gsutil cp {filename} {landing_path}

def on_open(ws):
    subscribe_message = {
        "type": "subscribe",
        "channels": [{"name": "ticker", "product_ids": ["BTC-USD", "ETH-USD", "DOGE-USD"]}]
    }
    ws.send(json.dumps(subscribe_message))

ws = websocket.WebSocketApp(
    "wss://ws-feed.exchange.coinbase.com",
    on_open=on_open,
    on_message=on_message
)

ws.run_forever()